## Citations-

Deep LOB algorithm for raw only engineered mode is derived from-:  
Z. Zhang, S. Zohren and S. Roberts, "DeepLOB: Deep Convolutional Neural Networks for Limit Order Books," in IEEE Transactions on Signal Processing, vol. 67, no. 11, pp. 3001-3012, 1 June1, 2019, doi: 10.1109/TSP.2019.2907260.

Algorithm for the engineered features input mode is derived from-:     
Avraam Tsantekidis, Nikolaos Passalis, Anastasios Tefas, Juho Kanniainen, Moncef Gabbouj, Alexandros Iosifidis,
Using Deep Learning for price prediction by exploiting stationary limit order book features,
Applied Soft Computing,
Volume 93,
2020,
106401,
ISSN 1568-4946,
https://doi.org/10.1016/j.asoc.2020.106401.

In [1]:
import numpy as np, pandas as pd, torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CyclicLR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_float32_matmul_precision("high")
device

'cuda'

In [2]:
engineered_mode = "raw" #"raw" (DeepLOB) or "feat" (Tsantekidis et al.)
window = 50
stride = 10

# for 3-class labeling
k_smooth  = 5 # moving-average window (number of events) for past & future mid-price
alpha_bps = 0.1 # threshold alpha in basis points; tune to balance classes (e.g., 0.1 to 5.0)
h_fwd = 20

# Training epochs
epochs = 200

In [3]:
ask_p = [4*i for i in range(10)]
ask_v = [4*i+1 for i in range(10)]
bid_p = [4*i+2 for i in range(10)]
bid_v = [4*i+3 for i in range(10)]

ob = pd.read_csv("AMZN_2012-06-21_34200000_57600000_orderbook_10.csv", header=None)

a_px = ob[ask_p].add_prefix("ask_price")
a_sz = ob[ask_v].add_prefix("ask_size")
b_px = ob[bid_p].add_prefix("bid_price")
b_sz = ob[bid_v].add_prefix("bid_size")

mid = 0.5*(a_px.iloc[:,0] + b_px.iloc[:,0]).astype(float)
spread = a_px.iloc[:,0] - b_px.iloc[:,0]

In [4]:
ob

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,2239500,100,2231800,100,2239900,100,2230700,200,2240000,220,...,2202500,5000,2294300,100,2202000,100,2298000,100,2189700,100
1,2239500,100,2238100,21,2239900,100,2231800,100,2240000,220,...,2204000,100,2294300,100,2202500,5000,2298000,100,2202000,100
2,2239500,100,2238100,21,2239600,20,2231800,100,2239900,100,...,2204000,100,2267700,100,2202500,5000,2294300,100,2202000,100
3,2239500,100,2238100,21,2239600,20,2237500,100,2239900,100,...,2213000,4000,2267700,100,2204000,100,2294300,100,2202500,5000
4,2239500,100,2238100,21,2239600,20,2237500,100,2239900,100,...,2213000,4000,2267700,100,2204000,100,2294300,100,2202500,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269743,2206200,100,2205100,249,2206400,100,2205000,71,2206500,1290,...,2204300,2300,2207600,100,2204200,100,2207900,2300,2204100,3300
269744,2206400,100,2205100,249,2206500,1290,2205000,71,2206700,170,...,2204300,2300,2207900,2300,2204200,100,2208000,3100,2204100,3300
269745,2206400,100,2205100,249,2206500,1290,2205000,71,2206700,170,...,2204300,2300,2208000,3100,2204200,100,2208100,1700,2204100,3300
269746,2206300,100,2205100,249,2206400,100,2205000,71,2206500,1290,...,2204300,2300,2207900,2300,2204200,100,2208000,3100,2204100,3300


In [5]:
feat = pd.DataFrame(index=ob.index)
if engineered_mode == "feat":
    # stationary transforms
    feat["mid_ret"] = mid.pct_change().fillna(0.0)
    # relative prices vs mid(per level)
    for lvl in range(10):
        feat[f"ask_pdiff{lvl}"] = a_px.iloc[:,lvl] / mid - 1.0
        feat[f"bid_pdiff{lvl}"] = b_px.iloc[:,lvl] / mid - 1.0
    # depth aggregates
    for k in (1,3,5,10):
        feat[f"depth_ask{k}"] = a_sz.iloc[:,:k].sum(1)
        feat[f"depth_bid{k}"] = b_sz.iloc[:,:k].sum(1)
    # L1 imbalance
    feat["qi_lvl1"] = b_sz.iloc[:,0] / (a_sz.iloc[:,0] + b_sz.iloc[:,0] + 1e-9)

In [6]:
# creating X according to input mode
if engineered_mode == "raw":
    Xdf = pd.concat([a_px, a_sz, b_px, b_sz], axis=1)   # 40 columns
elif engineered_mode == "feat":
    Xdf = feat
else:
    raise ValueError("engineered_mode must be 'raw' or 'feat'")

In [7]:
# creating 3 class labels
# Moving averages of past and future mid-price
alpha = alpha_bps / 1e4  #(1 bps = 0.0001)

m_past = mid.shift(1).rolling(k_smooth, min_periods=1).mean()

m_fut  = mid.shift(-1).rolling(k_smooth, min_periods=1).mean()

m_fut_H = m_fut.shift(-(h_fwd-1))

m_base  = m_past.replace(0, np.nan).ffill().bfill()
delta   = (m_fut_H - m_past) / m_base

valid_mask = (~m_past.isna()) & (~m_fut_H.isna()) & (~delta.isna())
valid_idx  = mid.index[valid_mask]

# Build labels: 0=Down, 1=Neutral, 2=Up
y3 = pd.Series(1, index=valid_idx, dtype=np.int64)   # default Neutral
y3.loc[delta.loc[valid_idx] >  alpha] = 2
y3.loc[delta.loc[valid_idx] < -alpha] = 0

Xdf = Xdf.loc[valid_idx].copy()
print("Label distribution (Down/Neutral/Up):", y3.value_counts().sort_index().to_dict())
print(f"Kept {len(valid_idx)} rows after horizon/smoothing trimming.")

Label distribution (Down/Neutral/Up): {0: 100512, 1: 75583, 2: 93633}
Kept 269728 rows after horizon/smoothing trimming.


In [8]:
# Chronological split
split_idx = int(len(Xdf)*0.8)
X_train, X_test = Xdf.iloc[:split_idx], Xdf.iloc[split_idx:]
y_train, y_test = y3.iloc[:split_idx], y3.iloc[split_idx:]

# StandardScaler fit on train, transform both (no leakage)
scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=Xdf.columns, index=X_train.index)
X_test  = pd.DataFrame(scaler.transform(X_test),  columns=Xdf.columns, index=X_test.index)

In [9]:
class LOBDS3C(Dataset): # returns window dataset and label. label is assigned to window's end
    def __init__(self, X, y_int, window=50, stride=10):
        self.X = X.values.astype(np.float32)
        self.y = y_int.values.astype(np.int64)
        self.window = window; self.stride = stride
        self.idxs = list(range(0, len(X) - window, stride))
    def __len__(self): return len(self.idxs)
    def __getitem__(self, idx):
        s = self.idxs[idx]
        x = self.X[s:s+self.window]
        y = self.y[s + self.window - 1] # label at window end
        return torch.from_numpy(x), torch.tensor(y)

train_ds = LOBDS3C(X_train, y_train, window=window, stride=stride)
test_ds  = LOBDS3C(X_test,  y_test,  window=window, stride=stride)

train_loader = DataLoader(train_ds, batch_size=256, shuffle=True, drop_last=True)
test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False)

print(f"Train windows: {len(train_ds)} | Test windows: {len(test_ds)}")

Train windows: 21574 | Test windows: 5390


In [10]:
# Model creation (based on input mode)
class Inception(nn.Module): # time X feature inception mentioned in DeepLOB
    def __init__(self, c_in, c_out=32):
        super().__init__()
        self.b1 = nn.Conv2d(c_in, c_out, (1,1))
        self.b3 = nn.Conv2d(c_in, c_out, (1,3), padding=(0,1))
        self.b5 = nn.Conv2d(c_in, c_out, (1,5), padding=(0,2))
    def forward(self, x):
        return torch.cat([self.b1(x), self.b3(x), self.b5(x)], dim=1)

class DeepLOBRaw(nn.Module):
    def __init__(self, lstm_hidden=64, dropout_p=0.1):
        super().__init__()
        self.stem = nn.Conv2d(1, 32, (1,2), stride=(1,2))
        self.inc1 = Inception(32, c_out=32)
        self.inc2 = Inception(96, c_out=32)
        self.lstm = nn.LSTM(96, lstm_hidden, batch_first=True, bidirectional=True)
        self.att  = nn.Linear(2*lstm_hidden, 1)
        self.drop = nn.Dropout(dropout_p)
        self.out  = nn.Linear(2*lstm_hidden, 3)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = F.leaky_relu(self.stem(x))
        x = F.leaky_relu(self.inc1(x))
        x = F.leaky_relu(self.inc2(x))
        x = x.mean(-1).permute(0,2,1)
        h, _ = self.lstm(x)
        w = torch.softmax(self.att(h).squeeze(-1), dim=1)
        ctx = (h * w.unsqueeze(-1)).sum(1)
        ctx = self.drop(ctx)
        return self.out(ctx)

class FeatTCN(nn.Module):
    def __init__(self, f_in, lstm_hidden=64, dropout_p=0.1):
        super().__init__()
        self.tcn = nn.Sequential(
            nn.Conv1d(f_in, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.lstm = nn.LSTM(64, lstm_hidden, batch_first=True, bidirectional=True)
        self.att  = nn.Linear(2*lstm_hidden, 1)
        self.drop = nn.Dropout(dropout_p)
        self.out  = nn.Linear(2*lstm_hidden, 3)

    def forward(self, x):
        x = x.permute(0,2,1)
        x = self.tcn(x)
        x = x.permute(0,2,1)
        h, _ = self.lstm(x)
        w = torch.softmax(self.att(h).squeeze(-1), dim=1)
        ctx = (h * w.unsqueeze(-1)).sum(1)
        ctx = self.drop(ctx)
        return self.out(ctx)

# picking model based on mode
if engineered_mode == "raw":
    assert Xdf.shape[1] == 40, "Raw mode expects exactly 40 LOB columns (10 levels x {ap,as,bp,bv})."
    model = DeepLOBRaw(lstm_hidden=64, dropout_p=0.1).to(device)
elif engineered_mode == "feat":
    model = FeatTCN(f_in=Xdf.shape[1], lstm_hidden=64, dropout_p=0.1).to(device)

In [11]:
cls_counts = y_train.value_counts().reindex([0,1,2]).fillna(0).values.astype(np.float32)
cls_weights = torch.tensor((cls_counts.sum() / np.clip(cls_counts, 1, None)), device=device)

criterion = nn.CrossEntropyLoss(weight=cls_weights)
opt = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
sched = CyclicLR(opt, base_lr=3e-4, max_lr=1e-3,
                 step_size_up=max(1, len(train_loader)//2), cycle_momentum=False)

In [12]:
for epoch in range(1, epochs+1):
    model.train()
    tot, correct, run_loss = 0, 0, 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward(); opt.step(); sched.step()

        with torch.no_grad():
            pred = logits.argmax(1)
            correct += (pred == yb).sum().item()
            tot += yb.numel()
            run_loss += loss.item() * yb.numel()
    if epoch % 10 == 0:
        print(f"epoch {epoch:02d}  acc={correct/tot:.3f}  loss={run_loss/tot:.4f}")

epoch 10  acc=0.361  loss=1.0931
epoch 20  acc=0.394  loss=1.0849
epoch 30  acc=0.441  loss=1.0595
epoch 40  acc=0.481  loss=1.0251
epoch 50  acc=0.529  loss=0.9754
epoch 60  acc=0.569  loss=0.9300
epoch 70  acc=0.605  loss=0.8811
epoch 80  acc=0.636  loss=0.8329
epoch 90  acc=0.659  loss=0.7909
epoch 100  acc=0.676  loss=0.7567
epoch 110  acc=0.695  loss=0.7146
epoch 120  acc=0.718  loss=0.6791
epoch 130  acc=0.729  loss=0.6517
epoch 140  acc=0.748  loss=0.6142
epoch 150  acc=0.752  loss=0.5990
epoch 160  acc=0.767  loss=0.5642
epoch 170  acc=0.776  loss=0.5426
epoch 180  acc=0.792  loss=0.5080
epoch 190  acc=0.805  loss=0.4845
epoch 200  acc=0.808  loss=0.4728


In [13]:
model.eval()
probs_list, y_list = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        logits = model(xb.to(device)).cpu()
        probs  = torch.softmax(logits, dim=1)
        probs_list.append(probs)
        y_list.append(yb)
probs  = torch.cat(probs_list).numpy()
y_true = torch.cat(y_list).numpy()
y_pred = probs.argmax(1)

# Confusion matrix & precision/recall report
print("Confusion matrix (rows=true [Down,Neutral,Up], cols=pred):")
print(confusion_matrix(y_true, y_pred, labels=[0,1,2]))
print(classification_report(y_true, y_pred, labels=[0,1,2],
                            target_names=["Down","Neutral","Up"], digits=3))

Confusion matrix (rows=true [Down,Neutral,Up], cols=pred):
[[703 533 778]
 [536 396 525]
 [595 507 817]]
              precision    recall  f1-score   support

        Down      0.383     0.349     0.365      2014
     Neutral      0.276     0.272     0.274      1457
          Up      0.385     0.426     0.405      1919

    accuracy                          0.355      5390
   macro avg      0.348     0.349     0.348      5390
weighted avg      0.355     0.355     0.355      5390

